#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [121]:
import math

import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [122]:
!pip install folium

You should consider upgrading via the '/Users/wald3/.pyenv/versions/3.9.5/bin/python3.9 -m pip install --upgrade pip' command.


Загрузим данные из csv файла в датафрейм.

In [124]:
df = pd.read_csv('input_data.csv',delimiter=';')
df = df.loc[:100000]
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0


Отобразим на карте координаты наших построек.

In [125]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [126]:
# Функция для вычисления расстояния по формуле Хаверсина
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Радиус Земли в км
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2.0)**2 + \
        math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c  # Расстояние в км

# Координаты центра Москвы и Санкт-Петербурга
moscow_lat, moscow_lon = 55.7558, 37.6173
spb_lat, spb_lon = 59.9343, 30.3351

# Функция для применения к датафрейму
def calculate_distance(row, center_lat, center_lon):
    return haversine_distance(row['geo_lat'], row['geo_lon'], center_lat, center_lon)

# Создание признаков is_Moscow и is_Saint_Peterburg
df['distance_to_Moscow'] = df.apply(lambda row: calculate_distance(row, moscow_lat, moscow_lon), axis=1)
df['distance_to_Saint_Peterburg'] = df.apply(lambda row: calculate_distance(row, spb_lat, spb_lon), axis=1)

df['is_Moscow'] = df['distance_to_Moscow'] <= 20  # В радиусе 20 км от Москвы
df['is_Saint_Peterburg'] = df['distance_to_Saint_Peterburg'] <= 20  # В радиусе 20 км от СПб

# Преобразуем булевы значения в целочисленные
df['is_Moscow'] = df['is_Moscow'].astype(int)
df['is_Saint_Peterburg'] = df['is_Saint_Peterburg'].astype(int)

# Удаляем временные колонки расстояний
df = df.drop(columns=['distance_to_Moscow', 'distance_to_Saint_Peterburg'])

df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,street_id,id_region,house_id,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,56.780112,60.699355,0,2,620000.0,NaN,66,1632918.0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.540060,37.725112,3,0,142701.0,242543.0,50,681306.0,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.608154,40.138381,0,0,385000.0,NaN,1,NaN,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.738685,37.713668,3,2,353960.0,439378.0,23,1730985.0,0,0


**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [127]:
# Удаляем ненужные колонки
columns_to_drop = ['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id']
df = df.drop(columns=columns_to_drop)

df.head()

,date,price,level,levels,rooms,area,kitchen_area,building_type,is_Moscow,is_Saint_Peterburg
0,2021-01-01,2451300,15,31,1,30.3,0.0,0,0,0
1,2021-01-01,1450000,5,5,1,33.0,6.0,0,0,0
2,2021-01-01,10700000,4,13,3,85.0,12.0,3,0,0
3,2021-01-01,3100000,3,5,3,82.0,9.0,0,0,0
4,2021-01-01,2500000,2,3,1,30.0,9.0,3,0,0


**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: ['building_type', 'rooms', 'is_Moscow', 'is_Saint_Peterburg']

Числовые: ['price', 'level', 'levels', 'area', 'kitchen_area']

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [128]:
from sklearn.preprocessing import OneHotEncoder

# Определяем категориальные и числовые признаки
categorical_cols = ['building_type', 'rooms', 'is_Moscow', 'is_Saint_Peterburg']
numerical_cols = ['price', 'level', 'levels', 'area', 'kitchen_area']

# Инициализируем OneHotEncoder
ohe = OneHotEncoder(sparse=False, drop='first')  

# Применяем OneHotEncoding к категориальным признакам
ohe_features = ohe.fit_transform(df[categorical_cols])

# Получаем названия новых признаков после кодирования
ohe_feature_names = ohe.get_feature_names_out(categorical_cols)

# Создаем датафрейм с закодированными признаками
df_ohe = pd.DataFrame(ohe_features, columns=ohe_feature_names, index=df.index)

# Объединяем закодированные признаки с основным датафреймом и удаляем исходные категориальные колонки
df = pd.concat([df.drop(columns=categorical_cols), df_ohe], axis=1)

# Просмотр первых пяти строк обновленного датафрейма
df.head()

,date,price,level,levels,area,kitchen_area,building_type_1,building_type_2,building_type_3,building_type_4,...,rooms_2,rooms_3,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_9,is_Moscow_1,is_Saint_Peterburg_1
0,2021-01-01,2451300,15,31,30.3,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-01-01,1450000,5,5,33.0,6.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-01,10700000,4,13,85.0,12.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-01-01,3100000,3,5,82.0,9.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-01-01,2500000,2,3,30.0,9.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [129]:
# Преобразуем колонку 'date' в формат datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

# Вычисляем количество дней со дня первого наблюдения
first_date = df['date'].min()
df['days_since_first'] = (df['date'] - first_date).dt.days

# Добавляем признак отношения этажа к количеству этажей
df['floor_ratio'] = df['level'] / df['levels']

# Удаляем колонку 'date' из датафрейма
df = df.drop(columns=['date'])

# Просмотр первых пяти строк датафрейма после добавления новых признаков
df.head()

,price,level,levels,area,kitchen_area,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,...,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_9,is_Moscow_1,is_Saint_Peterburg_1,days_since_first,floor_ratio
0,2451300,15,31,30.3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.483871
1,1450000,5,5,33.0,6.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1.000000
2,10700000,4,13,85.0,12.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.307692
3,3100000,3,5,82.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.600000
4,2500000,2,3,30.0,9.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.666667


In [130]:
from sklearn.preprocessing import StandardScaler

# Определяем числовые признаки, которые необходимо нормализовать
numerical_features = ['level', 'levels', 'area', 'kitchen_area', 
                      'days_since_first', 'floor_ratio', 'price']

# Проверка на бесконечные значения в любых признаках
inf_mask = df.applymap(np.isinf)
print("Количество бесконечных значений в каждом признаке:")
print(inf_mask.sum())

# Проверка на пропущенные значения в любых признаках
nan_mask = df.isnull()
print("\nКоличество пропущенных значений в каждом признаке:")
print(nan_mask.sum())

# Замена бесконечных значений на NaN во всех признаках
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Проверка на наличие NaN после замены бесконечностей
print("\nКоличество пропущенных значений после замены бесконечностей:")
print(df.isnull().sum())

# Удаление всех строк с любыми NaN в любом из признаков
df_clean = df.dropna()

# Проверка размеров датафрейма до и после очистки
print(f"\nРазмер датафрейма до очистки: {df.shape}")
print(f"Размер датафрейма после очистки: {df_clean.shape}")

# Инициализируем StandardScaler
scaler = StandardScaler()

# Применяем масштабирование к числовым признакам в очищенном датафрейме
df_clean[numerical_features] = scaler.fit_transform(df_clean[numerical_features])

# Просмотр первых пяти строк очищенного и масштабированного датафрейма
df_clean.head()

Количество бесконечных значений в каждом признаке:
price                   0
level                   0
levels                  0
area                    0
kitchen_area            0
building_type_1         0
building_type_2         0
building_type_3         0
building_type_4         0
building_type_5         0
building_type_6         0
rooms_1                 0
rooms_2                 0
rooms_3                 0
rooms_4                 0
rooms_5                 0
rooms_6                 0
rooms_7                 0
rooms_8                 0
rooms_9                 0
is_Moscow_1             0
is_Saint_Peterburg_1    0
days_since_first        0
floor_ratio             5
dtype: int64

Количество пропущенных значений в каждом признаке:
price                   0
level                   0
levels                  0
area                    0
kitchen_area            0
building_type_1         0
building_type_2         0
building_type_3         0
building_type_4         0
building_type_5         0


/var/folders/jl/cjrfbx7n14g8kdszsn8y_st40000gn/T/ipykernel_81944/3416023821.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[numerical_features] = scaler.fit_transform(df_clean[numerical_features])


,price,level,levels,area,kitchen_area,building_type_1,building_type_2,building_type_3,building_type_4,building_type_5,...,rooms_4,rooms_5,rooms_6,rooms_7,rooms_8,rooms_9,is_Moscow_1,is_Saint_Peterburg_1,days_since_first,floor_ratio
0,-0.004877,1.618636,2.688352,-0.863139,0.199750,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.686908,-0.339428
1,-0.005375,-0.238932,-0.863877,-0.766438,0.352624,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.686908,1.404069
2,-0.000773,-0.424689,0.229117,1.095944,0.505497,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.686908,-0.934564
3,-0.004554,-0.610446,-0.863877,0.988499,0.429060,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.686908,0.052859
4,-0.004853,-0.796203,-1.137125,-0.873883,0.429060,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.686908,0.278060


**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [131]:
class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        """
        Инициализация KNNRegressor

        Параметры:
        - n_neighbors: количество соседей для учета при предсказании.
        - metric: метрика расстояния ('euclidean' поддерживается).
        """
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        """
        Запоминает тренировочные данные

        Параметры:
        - X: numpy.ndarray, признаки обучающей выборки.
        - y: numpy.ndarray, целевая переменная обучающей выборки.
        """
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        """
        Предсказывает значения для X

        Параметры:
        - X: numpy.ndarray, признаки тестовой выборки.

        Возвращает:
        - predictions: numpy.ndarray, предсказанные значения.
        """
        if self.X_train is None or self.y_train is None:
            raise Exception("Модель не обучена. Вызовите метод fit перед predict.")
        
        predictions = []
        for i, x in enumerate(X):
            # Вычисляем расстояния до всех тренировочных точек
            if self.metric == 'euclidean':
                distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
            else:
                raise ValueError("Поддерживаются только Евклидовы расстояния.")
            
            # Находим индексы k ближайших соседей
            neighbor_idxs = np.argsort(distances)[:self.n_neighbors]
            
            # Среднее значение целевой переменной среди соседей
            pred = np.mean(self.y_train[neighbor_idxs])
            predictions.append(pred)
        
        return np.array(predictions)

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [132]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        """
        Инициализация LinearRegression

        Параметры:
        - learning_rate: скорость обучения.
        - optimization: метод оптимизации ('SGD', 'Momentum', 'AdaGrad').
        - epsilon: маленькое число для предотвращения деления на ноль в AdaGrad.
        - decay_rate: коэффициент затухания для Momentum.
        - max_iter: максимальное количество итераций для обучения.
        """
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        # Для Momentum
        self.v_weights = None
        self.v_bias = None
        # Для AdaGrad
        self.G_weights = None
        self.G_bias = None

    def fit(self, X, y):
        """
        Обучение модели

        Параметры:
        - X: numpy.ndarray, матрица признаков (размерность: n_samples x n_features).
        - y: numpy.ndarray, целевая переменная (размерность: n_samples).
        """
        n_samples, n_features = X.shape
        # Инициализация весов и смещения
        self.weights = np.zeros(n_features)
        self.bias = 0

        if self.optimization == 'Momentum':
            self.v_weights = np.zeros(n_features)
            self.v_bias = 0
        elif self.optimization == 'AdaGrad':
            self.G_weights = np.zeros(n_features)
            self.G_bias = 0

        for i in range(self.max_iter):
            # Вычисление предсказаний
            y_pred = np.dot(X, self.weights) + self.bias
            # Вычисление ошибок
            error = y_pred - y
            # Вычисление градиентов
            dw = (2 / n_samples) * np.dot(X.T, error)
            db = (2 / n_samples) * np.sum(error)

            if self.optimization == 'SGD':
                # Обновление весов и смещения для SGD
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

            elif self.optimization == 'Momentum':
                # Обновление весов и смещения для Momentum
                self.v_weights = self.decay_rate * self.v_weights + (1 - self.decay_rate) * dw
                self.v_bias = self.decay_rate * self.v_bias + (1 - self.decay_rate) * db
                self.weights -= self.learning_rate * self.v_weights
                self.bias -= self.learning_rate * self.v_bias

            elif self.optimization == 'AdaGrad':
                # Обновление весов и смещения для AdaGrad
                self.G_weights += dw ** 2
                self.G_bias += db ** 2
                adjusted_lr_w = self.learning_rate / (np.sqrt(self.G_weights) + self.epsilon)
                adjusted_lr_b = self.learning_rate / (np.sqrt(self.G_bias) + self.epsilon)
                self.weights -= adjusted_lr_w * dw
                self.bias -= adjusted_lr_b * db

            else:
                raise ValueError("Поддерживаются только оптимизаторы 'SGD', 'Momentum', 'AdaGrad'.")


    def predict(self, X):
        """
        Предсказание значений

        Параметры:
        - X: numpy.ndarray, матрица признаков (размерность: n_samples x n_features).

        Возвращает:
        - y_pred: numpy.ndarray, предсказанные значения (размерность: n_samples).
        """
        return np.dot(X, self.weights) + self.bias

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [133]:
from sklearn.model_selection import train_test_split

# Разделение данных на признаки X и целевую переменную Y
X = df_clean.drop(columns=['price'])
y = df_clean['price'].values

# Разделение на обучающую и тестовую выборки (80% - обучение, 20% - тест)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Просмотр размеров выборок
print(f"Размер обучающей выборки: {X_train.shape[0]} примеров")
print(f"Размер тестовой выборки: {X_test.shape[0]} примеров")

Размер обучающей выборки: 79996 примеров
Размер тестовой выборки: 19999 примеров


2. Обучение собственных моделей KNNRegressor и LinearRegression

2.1. Обучение собственной модели KNNRegressor

In [134]:
# Преобразование датафреймов в numpy массивы для собственной реализации
X_train_np = X_train.values
X_test_np = X_test.values

# Инициализация и обучение собственной модели KNNRegressor
knn_custom = KNNRegressor(n_neighbors=5, metric='euclidean')
knn_custom.fit(X_train_np, y_train)

# Предсказание на тестовой выборке
y_pred_knn_custom = knn_custom.predict(X_test_np)

2.2. Обучение собственной модели LinearRegression с разными оптимизаторами

In [135]:
# Инициализация и обучение собственной модели LinearRegression с оптимизатором SGD
lr_sgd = LinearRegression(
    learning_rate=0.01,
    optimization='SGD',
    max_iter=1000
)
lr_sgd.fit(X_train_np, y_train)
y_pred_lr_sgd = lr_sgd.predict(X_test_np)

# Инициализация и обучение собственной модели LinearRegression с оптимизатором Momentum
lr_momentum = LinearRegression(
    learning_rate=0.01,
    optimization='Momentum',
    decay_rate=0.9,
    max_iter=1000
)
lr_momentum.fit(X_train_np, y_train)
y_pred_lr_momentum = lr_momentum.predict(X_test_np)

# Инициализация и обучение собственной модели LinearRegression с оптимизатором AdaGrad
lr_adagrad = LinearRegression(
    learning_rate=0.01,
    optimization='AdaGrad',
    epsilon=1e-8,
    max_iter=1000
)
lr_adagrad.fit(X_train_np, y_train)
y_pred_lr_adagrad = lr_adagrad.predict(X_test_np)

3. Обучение библиотечных моделей из scikit-learn

3.1. Обучение библиотеки KNeighborsRegressor

In [136]:
from sklearn.neighbors import KNeighborsRegressor as SKKNeighborsRegressor

# Инициализация и обучение библиотечной модели KNeighborsRegressor
knn_sklearn = SKKNeighborsRegressor(n_neighbors=5, metric='euclidean')
knn_sklearn.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_knn_sklearn = knn_sklearn.predict(X_test)

3.2. Обучение библиотеки LinearRegression

In [137]:
from sklearn.linear_model import LinearRegression as SKLinearRegression

# Инициализация и обучение библиотечной модели LinearRegression
lr_sklearn = SKLinearRegression()
lr_sklearn.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_lr_sklearn = lr_sklearn.predict(X_test)

4. Вычисление метрик качества

Для оценки качества моделей мы будем использовать следующие метрики:

	•	MSE (Mean Squared Error): Среднеквадратичная ошибка.
	•	MAE (Mean Absolute Error): Средняя абсолютная ошибка.
	•	RMSE (Root Mean Squared Error): Корень из среднеквадратичной ошибки.

In [138]:
import pandas as pd

# Создание списка моделей и их предсказаний
models = {
    "Собственный KNNRegressor": y_pred_knn_custom,
    "Собственный LinearRegression (SGD)": y_pred_lr_sgd,
    "Собственный LinearRegression (Momentum)": y_pred_lr_momentum,
    "Собственный LinearRegression (AdaGrad)": y_pred_lr_adagrad,
    "Sklearn KNeighborsRegressor": y_pred_knn_sklearn,
    "Sklearn LinearRegression": y_pred_lr_sklearn
}

# Инициализация списка для хранения метрик
metrics_list = []

# Вычисление метрик для каждой модели
for model_name, y_pred in models.items():
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    metrics_list.append({
        "Модель": model_name,
        "MSE": mse,
        "MAE": mae,
        "RMSE": rmse
    })

# Создание DataFrame с метриками
metrics_df = pd.DataFrame(metrics_list)

# Сортировка по MSE для лучшей наглядности
metrics_df = metrics_df.sort_values(by="MSE").reset_index(drop=True)

# Отображение таблицы метрик
display(metrics_df)

,Модель,MSE,MAE,RMSE
0,Sklearn LinearRegression,4.998797,0.017434,2.235799
1,Собственный LinearRegression (AdaGrad),4.998797,0.017434,2.235799
2,Собственный LinearRegression (Momentum),4.998828,0.017400,2.235806
3,Собственный LinearRegression (SGD),4.998828,0.017399,2.235806
4,Собственный KNNRegressor,4.999077,0.016845,2.235862
5,Sklearn KNeighborsRegressor,4.999077,0.016845,2.235862


# Выводы

1. **Корректность реализации моделей:**
    Собственные реализации классов `LinearRegression` и `KNNRegressor` демонстрируют метрики, практически идентичные библиотечным моделям из `scikit-learn`, что говорит о корректности нашей реализации;

2. **Сравнение собственных и библиотечных моделей:**
   **Линейная регрессия:** Собственные реализации с оптимизаторами `AdaGrad`, `Momentum`, `SGD` показывают одинаковую производительность с библиотечной моделью;
   **KNN регрессия:** Наша реализация полностью совпадает с библиотечной моделью по всем метрикам.

3. **Оптимизаторы в линейной Регрессии:**
   `AdaGrad`, `Momentum`, и `SGD` показывают похожую производительность, что говорит о правильной реализации градиентного спуска и его вариантов.